# Tech Challenge Fase 4 — Entendendo o código (passo a passo)

Este notebook foi feito para você **rodar e entender** o que cada parte do projeto faz antes de gravar o vídeo.  
A ideia é conectar **teoria → código → resultado**, mostrando por que cada etapa existe.

**O que você vai ver aqui:**
1. Estrutura do projeto (onde está cada coisa e como os módulos se conversam)
2. Dados (Obesity.csv) + lógica de tratamento (ruído, discretização e dicionário)
3. Feature engineering (IMC/BMI) e base traduzida PT-BR
4. Treinamento do modelo (pipeline, balanceamento e métricas)
5. Como o Streamlit consome o modelo (Predicao) e como o Dashboard gera os gráficos
6. Dicas de deploy (paths no Streamlit Cloud)

**Conceitos-chave trabalhados:**
- EDA e análise descritiva para entender a distribuição das classes
- Engenharia de atributos para agregar informação (IMC)
- Pré-processamento e prevenção de *data leakage*
- Avaliação de um problema multiclasse com métricas adequadas


## 0) Setup rápido

> **Importante:** rode este notebook a partir da pasta `obesity_tc_project/` (ou ajuste o `PROJECT_DIR` abaixo).

O objetivo do setup é garantir que:
- o Python consiga importar o pacote local `src.obesity_tc`
- os caminhos para `data/`, `models/` e `reports/` resolvam corretamente
- o notebook seja reproduzível em qualquer computador (basta ajustar `PROJECT_DIR`)

Ajustar o diretório-base é uma boa prática em projetos de ciência de dados, porque evita caminhos absolutos rígidos
que quebram quando o projeto muda de máquina ou é levado para a nuvem.


In [ ]:
from pathlib import Path
import sys

# Ajuste este caminho para a pasta do projeto no seu computador.
# Exemplo Windows: PROJECT_DIR = Path(r"C:\...\tech4challenge\obesity_tc_project")
# Exemplo Linux/Mac: PROJECT_DIR = Path("/.../tech4challenge/obesity_tc_project")
PROJECT_DIR = Path(".").resolve()

# Garante que `src/` do projeto está no sys.path (para importar src.obesity_tc.*)
SRC_DIR = PROJECT_DIR / "src"
if str(PROJECT_DIR) not in sys.path:
    sys.path.insert(0, str(PROJECT_DIR))
if str(SRC_DIR) not in sys.path:
    sys.path.insert(0, str(SRC_DIR))

PROJECT_DIR


## 1) Estrutura do projeto

Os arquivos mais importantes e a função de cada um no fluxo:

- `Predicao.py`: **app principal** do Streamlit (formulário + predição)
- `pages/1_Dashboard.py`: **dashboard analítico** (EDA + insights)
- `pages/3_Metricas.py`: **documentação e métricas** do modelo
- `src/obesity_tc/make_dataset.py`: **tratamento dos dados** (limpeza, arredondamento, IMC, tradução)
- `src/obesity_tc/train.py`: **treinamento** (pipeline, SMOTE, salva modelo e relatórios)
- `data/raw/Obesity.csv`: base original
- `data/processed/base_traduzida_ptbr.csv`: base tratada + traduzida
- `models/modelo_obesidade.joblib`: modelo treinado
- `reports/metrics.json`: métricas do treino

**Fluxo conceitual do projeto:**
1. Dados brutos → tratamento (`make_dataset.py`)
2. Treino e avaliação → modelo + relatórios (`train.py`)
3. Consumo do modelo em produção → Streamlit (`Predicao.py` + `pages/*`)


In [ ]:
import os

def listar_pasta(base: Path, max_depth: int = 3):
    base = base.resolve()
    for root, dirs, files in os.walk(base):
        depth = Path(root).relative_to(base).parts
        if len(depth) > max_depth:
            continue
        indent = "  " * len(depth)
        print(f"{indent}{Path(root).name}/")
        for f in sorted(files):
            print(f"{indent}  - {f}")

listar_pasta(PROJECT_DIR, max_depth=3)


## 2) Carregando os dados (raw) e o dicionário

A base `Obesity.csv` traz variáveis numéricas e categóricas sobre perfil e hábitos de vida.  
Algumas variáveis são **discretas por natureza**, mas chegam com decimais (ruído de coleta ou conversão).  
Por isso, o tratamento arredonda colunas como `FCVC`, `NCP`, `CH2O`, `FAF`, `TUE` antes de interpretar categorias.

Depois, criamos uma feature essencial: **IMC/BMI = Weight / Height²**.

Do ponto de vista teórico:
- arredondar variáveis discretas evita criar "categorias falsas" (ex.: 2.0, 2.1, 2.2)
- o IMC sintetiza **peso e altura** em uma medida comparável entre indivíduos
- a base traduzida melhora a comunicação, mas não altera a informação em si


In [ ]:
import pandas as pd

RAW_PATH = PROJECT_DIR / "data" / "raw" / "Obesity.csv"
df_raw = pd.read_csv(RAW_PATH)

df_raw.shape, df_raw.head(3)


### 2.1) Conferindo “ruído decimal” nas variáveis discretas

Aqui você verifica se variáveis discretas aparecem com decimais.  
Essa checagem é importante porque, em modelos de ML, valores contínuos e discretos são tratados de formas diferentes.

Se uma variável deveria ter poucos valores (ex.: 1, 2, 3) e aparece como 1.7 ou 2.4, o modelo pode:
- interpretar como escala contínua e aprender relações inexistentes
- criar separações artificiais, aumentando ruído

Por isso, o tratamento de discretização é uma etapa crítica antes do treinamento.


In [ ]:
import numpy as np

cols_discretas = ["FCVC","NCP","CH2O","FAF","TUE"]
for c in cols_discretas:
    if c in df_raw.columns:
        amostra = df_raw[c].dropna().astype(float).head(10).tolist()
        tem_decimal = any(abs(x - round(x)) > 1e-9 for x in amostra)
        print(c, "exemplo:", amostra[:5], "| tem decimal?", tem_decimal)


## 3) Tratamento dos dados: `make_dataset.py`

Este módulo centraliza o *pré-processamento* para garantir consistência. Ele faz:
1. **Limpeza de strings** (remove espaços extras, padroniza textos)
2. **Arredondamento** de colunas discretas com ruído
3. **Cálculo do IMC** para enriquecer o dataset
4. **Tradução PT-BR** para dashboards e comunicação com usuários

A ideia é que o mesmo tratamento seja usado no treino e na predição, evitando
problemas de inconsistência entre dados de treinamento e de produção.


In [ ]:
from src.obesity_tc.make_dataset import preprocessar_base, traduzir_ptbr

df_proc = preprocessar_base(df_raw, coluna_alvo="Obesity")
df_ptbr = traduzir_ptbr(df_proc)

df_proc.shape, df_proc.head(3)


### 3.1) Validando o IMC (BMI)

O IMC é uma variável amplamente utilizada em saúde pública porque normaliza o peso pela altura.
Ele ajuda a capturar **massa corporal relativa**, o que é mais informativo do que peso isolado.

**Fórmula:** `IMC = peso (kg) / altura (m)^2`

Por que isso importa?
- duas pessoas com o mesmo peso podem ter alturas diferentes
- o IMC ajuda a distinguir melhor perfis corporais
- o modelo consegue separar classes de obesidade com mais clareza

Observação: o IMC é um indicador estatístico, não um diagnóstico clínico isolado.


In [ ]:
df_proc["BMI"].describe().round(2)


### 3.2) Base traduzida (PT-BR)

A tradução é usada para **visualização e comunicação** (dashboard e relatório).  
Ela não altera a informação, apenas troca rótulos para português.

Isso é importante porque:
- facilita a explicação no vídeo e na apresentação
- reduz ambiguidade para o público não técnico
- torna gráficos e tabelas mais legíveis


In [ ]:
df_ptbr[["Gênero","Idade","Altura (m)","Peso (kg)","IMC","Nível de obesidade"]].head(5)


## 4) Treinamento do modelo: `train.py`

O treinamento é um problema **multiclasse supervisionado**, com o objetivo de prever o nível de obesidade.
No treino você monta um pipeline com:
- `ColumnTransformer` para separar numéricas e categóricas
- `OneHotEncoder` nas categorias (transforma texto em vetores)
- `MinMaxScaler` nas numéricas (normaliza escalas)
- `SMOTE` para balancear classes
- `RandomForestClassifier` como modelo final

Você salva:
- `models/modelo_obesidade.joblib`
- `reports/metrics.json` + `classification_report.txt`

Do ponto de vista teórico, o pipeline garante que **o mesmo tratamento** seja aplicado no treino e na predição,
reduzindo o risco de *data leakage* e inconsistências.


In [ ]:
import json
from joblib import load

MODEL_PATH = PROJECT_DIR / "models" / "modelo_obesidade.joblib"
METRICS_PATH = PROJECT_DIR / "reports" / "metrics.json"

bundle = load(MODEL_PATH)
pipeline = bundle["pipeline"] if isinstance(bundle, dict) and "pipeline" in bundle else bundle
metrics = json.loads(METRICS_PATH.read_text(encoding="utf-8"))

type(pipeline), metrics["acuracia"]


### 4.1) Inspecionando o pipeline (SMOTE + RF)

O pipeline organiza a sequência de transformações e o modelo final.  
Essa organização é essencial porque garante que cada etapa aconteça sempre na mesma ordem.

**Ordem típica:**
1. Separar numéricas/categóricas
2. Normalizar numéricas (MinMax)
3. Codificar categorias (OneHot)
4. Balancear classes no treino (SMOTE)
5. Treinar o Random Forest

O Random Forest funciona como um conjunto de árvores de decisão, reduzindo variância e melhorando robustez.


In [ ]:
pipeline


### 4.2) Matriz de confusão e classes

A matriz de confusão mostra **onde o modelo acerta e erra** por classe.  
Ela é especialmente útil em problemas multiclasse, porque revela confusões específicas
(ex.: Sobrepeso II confundido com Obesidade I).

No arquivo `metrics.json`, a matriz já vem pronta, calculada no conjunto de teste.
Isso permite analisar o desempenho sem reexecutar todo o treino.


In [ ]:
import numpy as np
import pandas as pd

classes = metrics["classes"]
cm = np.array(metrics["matriz_confusao"])

pd.DataFrame(cm, index=classes, columns=classes)


## 5) Como a predição funciona no Streamlit (`Predicao.py`)

Fluxo do app:
1. Carrega o modelo (`modelo_obesidade.joblib`)
2. (Se necessário) atualiza `base_traduzida_ptbr.csv` para o dashboard
3. Mostra formulário com inputs (idade, altura, peso, hábitos…)
4. Converte inputs para o formato que o modelo espera (labels originais)
5. Faz `pipeline.predict()` e retorna a classe prevista

Do ponto de vista teórico:
- o modelo precisa receber **as mesmas variáveis** e o mesmo tratamento do treino
- o pipeline encapsula isso para evitar erro humano
- a predição é probabilística e serve como apoio à decisão, não como diagnóstico clínico

Vamos simular uma predição aqui no notebook usando uma linha da base.


In [ ]:
# separa X e y
X = df_proc.drop(columns=["Obesity_level"])
y = df_proc["Obesity_level"]

# pega um exemplo
linha = X.iloc[[0]].copy()
pred = pipeline.predict(linha)[0]

pred, y.iloc[0]


## 6) Dashboard analítico — reproduzindo os gráficos fora do Streamlit

Aqui recriamos os mesmos gráficos do `pages/1_Dashboard.py`, mas dentro do notebook.  
Isso ajuda a explicar **o raciocínio estatístico** por trás de cada visualização.

O objetivo do dashboard é:
- entender a distribuição das classes
- verificar relações entre hábitos e obesidade
- gerar insights visuais para a apresentação


In [ ]:
import plotly.express as px

# Usaremos a base traduzida para ficar igual ao dashboard
df = df_ptbr.copy()

df.columns


### 6.1) KPIs iniciais (cards)

Esses KPIs resumem a base com estatísticas simples:
- **Registros:** tamanho do dataset
- **Idade média / IMC médio / Peso médio:** medidas de tendência central

São úteis para dar contexto rápido antes de mostrar gráficos detalhados.


In [ ]:
kpis = {
    "Registros": len(df),
    "Idade média": df["Idade"].mean(),
    "IMC médio": df["IMC"].mean(),
    "Peso médio (kg)": df["Peso (kg)"].mean(),
}
{k: round(v, 1) if isinstance(v, float) else v for k,v in kpis.items()}


### 6.2) Distribuição dos níveis de obesidade (pizza)

Mostra o "mix" das classes no dataset (alvo multiclasse).  
Essa visão é importante porque revela **desbalanceamentos** que podem enviesar o modelo.

Se uma classe é muito rara, o algoritmo tende a errar mais nela, o que justifica o uso do SMOTE.


In [ ]:
fig = px.pie(df, names="Nível de obesidade", title="Distribuição dos níveis de obesidade")
fig.show()


### 6.3) Distribuição por gênero

Checa se o dataset está balanceado entre Masculino/Feminino.  
Um desbalanceamento forte pode indicar viés amostral ou necessidade de estratificação.

Também ajuda a interpretar se o comportamento do modelo varia por subgrupos.


In [ ]:
fig = px.bar(df, x="Gênero", title="Distribuição por gênero")
fig.show()


### 6.4) Modos de transporte (MTRANS)

O meio de transporte é um **proxy de estilo de vida**:
- transporte público/carro podem indicar menor atividade diária
- caminhada/bicicleta podem indicar maior gasto calórico

Como variável categórica, ela ajuda o modelo a capturar padrões de hábitos e mobilidade.


In [ ]:
fig = px.bar(
    df, x="Meio de transporte", title="Uso de modos de transporte", labels={"Meio de transporte":"Meio de transporte","count":"Quantidade"}
)
fig.show()


### 6.5) Relação entre altura e peso (colorido por classe)

Essa visualização mostra como as classes se distribuem no plano altura × peso.  
A separação entre cores sugere que existe padrão claro de classe, o que justifica o uso do IMC.

Se as classes se sobrepõem muito, o modelo precisa de variáveis adicionais para diferenciar perfis.


In [ ]:
fig = px.scatter(
    df,
    x="Altura (m)",
    y="Peso (kg)",
    color="Nível de obesidade",
    title="Relação entre altura e peso",
    opacity=0.8
)
fig.show()


### 6.6) Correlação entre variáveis numéricas

A correlação (geralmente de Pearson) mede **associação linear** entre duas variáveis.

Por que isso importa?
- ajuda a detectar variáveis redundantes
- explica por que o IMC tem alta correlação com peso
- não implica causalidade, mas aponta relações úteis

Esse gráfico é uma boa ponte entre teoria estatística e interpretação prática.


In [ ]:
import numpy as np

num_cols = ["Idade","Altura (m)","Peso (kg)","Consumo de vegetais","Refeições principais","Consumo de água","Atividade física","Tempo usando tecnologia","IMC"]
corr = df[num_cols].corr()

fig = px.imshow(
    corr.round(2),
    text_auto=True,
    title="Correlação entre variáveis numéricas",
    aspect="auto"
)
fig.show()

corr.loc["IMC","Peso (kg)"]


### 6.7) Histórico familiar por nível de obesidade

Aqui avaliamos o fator hereditário.  
Se a classe "Obesidade tipo III" aparece concentrada em pessoas com histórico familiar,
isso sugere um sinal forte que o modelo pode explorar.

A tabela cruzada ajuda a quantificar essa relação.


In [ ]:
fig = px.histogram(
    df,
    x="Histórico familiar de sobrepeso",
    color="Nível de obesidade",
    barmode="group",
    title="Histórico familiar de sobrepeso por nível de obesidade"
)
fig.show()

import pandas as pd
pd.crosstab(df["Nível de obesidade"], df["Histórico familiar de sobrepeso"])


### 6.8) Hidratação (CH2O) e monitoramento de calorias (SCC)

Dois comportamentos importantes de saúde:
- **CH2O:** quantidade de água ingerida por dia
- **SCC:** hábito de monitorar calorias

São variáveis comportamentais e podem ter relação com padrões de obesidade.


In [ ]:
fig = px.histogram(
    df,
    x="Consumo de água",
    color="Nível de obesidade",
    barmode="group",
    title="Consumo de água (CH2O) por nível de obesidade",
)
fig.show()

fig2 = px.histogram(
    df,
    x="Monitoramento de calorias",
    color="Nível de obesidade",
    barmode="group",
    title="Monitoramento de calorias (SCC) por nível de obesidade",
)
fig2.show()

df["Monitoramento de calorias"].value_counts(normalize=True).round(3)


### 6.9) Atividade física (FAF) e tempo usando tecnologia (TUE)

Essas variáveis funcionam como indicadores de sedentarismo vs. atividade:
- FAF mede frequência/intensidade de exercício
- TUE indica tempo diário em atividades sedentárias (tela/tecnologia)

É comum observar correlação inversa entre atividade física e níveis de obesidade.


In [ ]:
fig = px.histogram(
    df,
    x="Atividade física",
    color="Nível de obesidade",
    barmode="group",
    title="Atividade física (FAF) por nível de obesidade",
)
fig.show()

fig2 = px.histogram(
    df,
    x="Tempo usando tecnologia",
    color="Nível de obesidade",
    barmode="group",
    title="Tempo usando tecnologia (TUE) por nível de obesidade",
)
fig2.show()


### 6.10) Consumo e refeições (NCP + álcool)

- **NCP:** número de refeições principais por dia
- **CALC:** consumo de álcool

O padrão alimentar influencia diretamente o balanço energético, e o álcool adiciona calorias extras.
Esses gráficos ajudam a mostrar como hábitos alimentares se distribuem por classe.


In [ ]:
fig = px.histogram(
    df,
    x="Refeições principais",
    color="Nível de obesidade",
    barmode="group",
    title="Número de refeições principais (NCP)",
)
fig.show()

fig2 = px.histogram(
    df,
    x="Consumo de álcool",
    color="Nível de obesidade",
    barmode="group",
    title="Consumo de álcool (CALC)",
)
fig2.show()


### 6.11) Hábitos alimentares (FAVC + CAEC)

- **FAVC:** consumo frequente de alimentos hipercalóricos
- **CAEC:** beliscar entre refeições

Ambos estão ligados a maior ingestão calórica.  
Quando combinados, podem explicar diferenças relevantes entre classes de obesidade.


In [ ]:
fig = px.histogram(
    df,
    x="Alimentos hipercalóricos frequentes",
    color="Nível de obesidade",
    barmode="group",
    title="Consumo de alimentos hipercalóricos (FAVC)",
)
fig.show()

fig2 = px.histogram(
    df,
    x="Beliscar entre refeições",
    color="Nível de obesidade",
    barmode="group",
    title="Beliscar entre refeições (CAEC)",
)
fig2.show()


### 6.12) Idade e IMC (distribuições)

Encerramos o dashboard com duas distribuições-chave:
- **Idade:** mostra se há concentração de classes em faixas etárias específicas
- **IMC:** resume diretamente a variável central do problema

Essas distribuições ajudam a reforçar a narrativa sobre o comportamento dos dados.


In [ ]:
fig = px.histogram(
    df,
    x="Idade",
    color="Nível de obesidade",
    barmode="overlay",
    opacity=0.5,
    nbins=25,
    title="Distribuição de idade por nível de obesidade",
)
fig.show()

fig2 = px.histogram(
    df,
    x="IMC",
    color="Nível de obesidade",
    barmode="overlay",
    opacity=0.5,
    nbins=30,
    title="Distribuição de IMC por nível de obesidade",
)
fig2.show()


## 7) Dica final de deploy (Streamlit Cloud)

O Streamlit Cloud roda com *working directory* na raiz do repo.  
Por isso, no seu código você usa:

- `BASE_DIR = Path(__file__).resolve().parent` (em `Predicao.py`)
- `BASE_DIR = Path(__file__).resolve().parents[1]` (em `pages/*`)

Isso garante que `data/`, `models/` e `reports/` sejam encontrados de forma consistente,
independente de onde o app é executado.
